# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 2:54PM,241121,10,6676034,"Senseonics, Incorporated",Released
1,Jul 5 2022 2:50PM,241120,19,ADV80003801,"AdvaGen Pharma, Ltd",Released
2,Jul 5 2022 2:50PM,241120,19,ADV80003785,"AdvaGen Pharma, Ltd",Released
3,Jul 5 2022 2:50PM,241120,19,ADV80003788,"AdvaGen Pharma, Ltd",Released
4,Jul 5 2022 2:50PM,241120,19,ADV80003789,"AdvaGen Pharma, Ltd",Released
5,Jul 5 2022 2:50PM,241120,19,ADV80003790,"AdvaGen Pharma, Ltd",Released
6,Jul 5 2022 2:50PM,241120,19,ADV80003791,"AdvaGen Pharma, Ltd",Released
7,Jul 5 2022 2:50PM,241120,19,ADV80003792,"AdvaGen Pharma, Ltd",Released
8,Jul 5 2022 2:50PM,241120,19,ADV80003793,"AdvaGen Pharma, Ltd",Released
9,Jul 5 2022 2:50PM,241120,19,ADV80003794,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
66,241117,Released,1
67,241118,Released,8
68,241119,Released,1
69,241120,Released,12
70,241121,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241117,NaN,NaN,1.0
241118,NaN,NaN,8.0
241119,NaN,NaN,1.0
241120,NaN,NaN,12.0
241121,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241010,0.0,0.0,60.0
241016,0.0,0.0,60.0
241017,0.0,0.0,60.0
241019,0.0,0.0,55.0
241022,59.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241010,0,0,60
241016,0,0,60
241017,0,0,60
241019,0,0,55
241022,59,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241010,0,0,60
1,241016,0,0,60
2,241017,0,0,60
3,241019,0,0,55
4,241022,59,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241010,,,60
1,241016,,,60
2,241017,,,60
3,241019,,,55
4,241022,59,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 2:54PM,241121,10,"Senseonics, Incorporated"
1,Jul 5 2022 2:50PM,241120,19,"AdvaGen Pharma, Ltd"
13,Jul 5 2022 2:45PM,241119,16,Seqirus
14,Jul 5 2022 2:43PM,241118,10,ISDIN Corporation
22,Jul 5 2022 2:37PM,241117,21,"NBTY Global, Inc."
23,Jul 5 2022 2:36PM,241110,10,"Methapharm, Inc."
31,Jul 5 2022 2:35PM,241116,19,"AdvaGen Pharma, Ltd"
54,Jul 5 2022 2:31PM,241115,10,Emerginnova
58,Jul 5 2022 2:27PM,241113,10,"Senseonics, Incorporated"
59,Jul 5 2022 1:58PM,241112,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 5 2022 2:54PM,241121,10,"Senseonics, Incorporated",,,1
1,Jul 5 2022 2:50PM,241120,19,"AdvaGen Pharma, Ltd",,,12
2,Jul 5 2022 2:45PM,241119,16,Seqirus,,,1
3,Jul 5 2022 2:43PM,241118,10,ISDIN Corporation,,,8
4,Jul 5 2022 2:37PM,241117,21,"NBTY Global, Inc.",,,1
5,Jul 5 2022 2:36PM,241110,10,"Methapharm, Inc.",,,8
6,Jul 5 2022 2:35PM,241116,19,"AdvaGen Pharma, Ltd",,,23
7,Jul 5 2022 2:31PM,241115,10,Emerginnova,,,4
8,Jul 5 2022 2:27PM,241113,10,"Senseonics, Incorporated",,,1
9,Jul 5 2022 1:58PM,241112,20,Alumier Labs Inc.,,,40


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 2:54PM,241121,10,"Senseonics, Incorporated",1,,
1,Jul 5 2022 2:50PM,241120,19,"AdvaGen Pharma, Ltd",12,,
2,Jul 5 2022 2:45PM,241119,16,Seqirus,1,,
3,Jul 5 2022 2:43PM,241118,10,ISDIN Corporation,8,,
4,Jul 5 2022 2:37PM,241117,21,"NBTY Global, Inc.",1,,
5,Jul 5 2022 2:36PM,241110,10,"Methapharm, Inc.",8,,
6,Jul 5 2022 2:35PM,241116,19,"AdvaGen Pharma, Ltd",23,,
7,Jul 5 2022 2:31PM,241115,10,Emerginnova,4,,
8,Jul 5 2022 2:27PM,241113,10,"Senseonics, Incorporated",1,,
9,Jul 5 2022 1:58PM,241112,20,Alumier Labs Inc.,40,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 2:54PM,241121,10,"Senseonics, Incorporated",1,,
1,Jul 5 2022 2:50PM,241120,19,"AdvaGen Pharma, Ltd",12,,
2,Jul 5 2022 2:45PM,241119,16,Seqirus,1,,
3,Jul 5 2022 2:43PM,241118,10,ISDIN Corporation,8,,
4,Jul 5 2022 2:37PM,241117,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 2:54PM,241121,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jul 5 2022 2:50PM,241120,19,"AdvaGen Pharma, Ltd",12.0,NaN,NaN
2,Jul 5 2022 2:45PM,241119,16,Seqirus,1.0,NaN,NaN
3,Jul 5 2022 2:43PM,241118,10,ISDIN Corporation,8.0,NaN,NaN
4,Jul 5 2022 2:37PM,241117,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 5 2022 2:54PM,241121,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jul 5 2022 2:50PM,241120,19,"AdvaGen Pharma, Ltd",12.0,0.0,0.0
2,Jul 5 2022 2:45PM,241119,16,Seqirus,1.0,0.0,0.0
3,Jul 5 2022 2:43PM,241118,10,ISDIN Corporation,8.0,0.0,0.0
4,Jul 5 2022 2:37PM,241117,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,10124625,1324.0,8.0,61.0
12,2169787,47.0,46.0,0.0
15,241073,11.0,8.0,0.0
16,482219,2.0,0.0,0.0
19,723293,35.0,1.0,0.0
20,1205453,108.0,20.0,12.0
21,241117,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,10124625,1324.0,8.0,61.0
1,12,2169787,47.0,46.0,0.0
2,15,241073,11.0,8.0,0.0
3,16,482219,2.0,0.0,0.0
4,19,723293,35.0,1.0,0.0
5,20,1205453,108.0,20.0,12.0
6,21,241117,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1324.0,8.0,61.0
1,12,47.0,46.0,0.0
2,15,11.0,8.0,0.0
3,16,2.0,0.0,0.0
4,19,35.0,1.0,0.0
5,20,108.0,20.0,12.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1324.0
1,12,Released,47.0
2,15,Released,11.0
3,16,Released,2.0
4,19,Released,35.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,61.0,0.0,0.0,0.0,0.0,12.0,0.0
Executing,8.0,46.0,8.0,0.0,1.0,20.0,0.0
Released,1324.0,47.0,11.0,2.0,35.0,108.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,61.0,0.0,0.0,0.0,0.0,12.0,0.0
1,Executing,8.0,46.0,8.0,0.0,1.0,20.0,0.0
2,Released,1324.0,47.0,11.0,2.0,35.0,108.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,61.0,0.0,0.0,0.0,0.0,12.0,0.0
1,Executing,8.0,46.0,8.0,0.0,1.0,20.0,0.0
2,Released,1324.0,47.0,11.0,2.0,35.0,108.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()